In [1]:
import torch 
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import time
import matplotlib.pyplot as plt


import torch
from torch.utils.data import DataLoader

from segmentation.config import CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, transforms
from segmentation.scr.utils.utils import set_seed, save_model
from segmentation.scr.tilling_dataset import Tilling_Dataset
from segmentation.scr.train_function import train_model
from segmentation.scr.utils.metrics import dice_coef

from colorama import Fore, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

#from segmentation.scr
pd.options.mode.chained_assignment = None

C:\Users\123\AppData\Local\Temp\ipykernel_15524\4064904260.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_transform  = transforms.get_transform(transform_type='weak')
val_transform = transforms.get_transform(transform_type='val')

In [2]:
train_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_till,
    use_random_sub=True,
    empty_tile_pct=30,
    sample_limit=7500,
    random_seed=CFG.random_seed
)


Dataset contains 20744 empty and 33952 non-empty tiles.
Sample 2250 empty and 5250 non-empty tiles.


In [3]:
val_dataset = Tilling_Dataset(
    name_data='kidney_3_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=40,
    sample_limit=2000,
    random_seed=CFG.random_seed)

Dataset contains 14595 empty and 6447 non-empty tiles.
Sample 800 empty and 1200 non-empty tiles.


In [4]:
set_seed(CFG.random_seed)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, num_workers=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(train_dataset, batch_size=CFG.valid_batch_size, num_workers=2, shuffle=False, pin_memory=True)

In [5]:
model = unet.UNet(n_channels=3, n_classes=1, bilinear=False).to(CFG.device)
num_epoch = 30
loss_fn = losses.BCE_DICE(mode="SUM")
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) 
sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode='max', patience=3, factor=0.6, verbose=True, threshold=1e-3)
device = CFG.device

In [14]:
train_model(model=model, 
                optimizer=optimizer, 
                loss_func=loss_fn, 
                train_loader=train_loader, 
                val_loader=val_loader, 
                num_epochs=num_epoch, 
                scheduler=sheduler, 
                device =CFG.device,
                path_to_save=CFG.path_to_save_state_model)

Epoch 1/30
----------


Valid :   8%|▊         | 36/469 [00:32<06:16,  1.15it/s, gpu_memory=5.29 GB, valid_loss=0.7862]